# 데이터 전처리

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import json
import glob

def load_json_files(path_pattern):
    files = glob.glob(path_pattern)
    data = []
    for file in tqdm(files, "Files"):
        with open(file, 'r', encoding='utf-8-sig') as f:
            json_data = json.load(f)
            data.append(json_data)
    return data

def create_one_hot_encoding(data):
    # 기본 데이터프레임 생성
    df = pd.DataFrame(data)
    saved = pd.DataFrame()
    
    # 원문 및 위치 정보 추가
    saved['문장'] = df['문장']

    return saved

# JSON 파일 로드
json_files_path = 'data/selectstar/C*.json'
data = load_json_files(json_files_path)

# 원본 데이터로부터 One-hot encoding DataFrame 생성
one_hot_df = create_one_hot_encoding(data)

# 결과 출력 및 CSV 파일로 저장(optional)
print(one_hot_df.head(n=10))
one_hot_df.to_csv('selectstar-encoded.csv', encoding='utf-8')

Files:   0%|          | 0/100000 [00:00<?, ?it/s]

                                                  문장
0  국민들은 깨어나야 합니다. 지금 민주당을 지지하는 사림들 보세요~ 종북주의자들이 판...
1                      저 어른들...지금 정부 ''꼬라지''' 보는것같구만
2  ''대깨문''' 출동 이번 코로나가 확산되는건 민노총 때문이라고 몰고가라~~~ 눼눼...
3                            아프간과 비교를...ㅎ ''치매드셨'''나
4  작년 국민집회때 #@이름#목사 감옥보내더니 ㅋㅋㅋ8000명 민노총은 집회허락하는 이...
5  태권도는 ''졸라''' ''깨지'''고~''창피하'''다!!!!!!태권도 종주국 이...
6  ''더불당'''은 동네 ''똥개''' 크게 ''소리''' ''짓'''는 ''암숫개'...
7  미국이 지원해서 가장 성공한 케이스가 대한민국.그리고 잘살게되니까 ''반미하'''는...
8                               세금 그렇게 먹으면 죽을만 하지 ㅇㅇ
9  또다시 더불어당 #@이름# #@이름#이 입을 맞추며 분위기를 잡고 있다. 싹 쓸어버...


In [2]:
import pandas as pd
import re

# 예시 DataFrame 생성
df = pd.read_csv("selectstar-encoded.csv", index_col=0)

def extract_profanity_positions(sentence):
    # '(욕설)' 형태로 마킹된 모든 욕설을 찾기 위한 정규 표현식
    pattern = r"''(.*?)'''"
    
    # 욕설 위치 저장할 리스트
    positions = []
    
    # 매칭된 모든 욕설에 대해 시작과 끝 인덱스를 계산
    for match in re.finditer(pattern, sentence):
        start = match.start(0)
        end = match.end(0) - 1  # end 인덱스는 포함하지 않으므로 -1
        positions.append((start, end))
        
    return positions

# 각 문장에 대해 시작/종료 인덱스 추가
df['욕설 위치'] = df['문장'].apply(extract_profanity_positions)
# 결과 출력
print(df[['문장', '욕설 위치']])
df.to_csv('./selectstar-location.csv', encoding="utf-8")

                                                      문장  \
0      국민들은 깨어나야 합니다. 지금 민주당을 지지하는 사림들 보세요~ 종북주의자들이 판...   
1                          저 어른들...지금 정부 ''꼬라지''' 보는것같구만   
2      ''대깨문''' 출동 이번 코로나가 확산되는건 민노총 때문이라고 몰고가라~~~ 눼눼...   
3                                아프간과 비교를...ㅎ ''치매드셨'''나   
4      작년 국민집회때 #@이름#목사 감옥보내더니 ㅋㅋㅋ8000명 민노총은 집회허락하는 이...   
...                                                  ...   
99995  \n판결 요약하면 ''기레기'''질은 맞는데 언론의 자유를 위해 봐준다임. 나쁜''...   
99996  나라를 위해 이렇게 몸 아끼지않고 헌신하는 애국자가 있는반면 파란지붕아래 ''쓰레기...   
99997  정권바뀌면 없어져야할것,,,,아무쓸모없고 25만원 받아야 아무도움안되니 세금이나 그...   
99998  그''놈'''의 #@이름#이가 뭐라고 저토록 지긋지긋하게 무죄가 만들어 주고싶어 그...   
99999  ''쌍욕달이''' ''깡패''' #@이름#과 ''망언''' ''전문''' ''문빠'...   

                                                   욕설 위치  
0                                           [(195, 203)]  
1                                             [(14, 21)]  
2                       [(0, 7), (129, 136), (229, 236)]  
3                                          

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

# 모델과 토크나이저 로드
model_name = "monologg/kobert"  # 또는 "roberta-base" 등
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)  # num_labels는 레이블 수
raw_dataset = pd.read_csv('./selectstar-location.csv')

def create_labels(sentence, positions):
    tokens = list(sentence)  # 문장을 문자 단위로 토큰화
    labels = ['O'] * len(tokens)  # 초기 레이블 설정

    if not positions:  # positions가 비어있는 경우
        return tokens, labels

    for start, end in positions:
        labels[start] = 'B-PROFANITY'
        for i in range(start + 1, end + 1):
            if i < len(labels):  # 인덱스가 범위를 벗어나지 않도록
                labels[i] = 'I-PROFANITY'

    return tokens, labels

# 데이터셋 생성하기
dataset = []
for index, row in raw_dataset.iterrows():
    tokens, labels = create_labels(row['문장'], row['욕설 위치'])
    dataset.append((tokens, labels))

texts, labels = zip(*dataset)

X_train, X_eval, Y_train, Y_eval = train_test_split(texts, labels, test_size=0.2)

# Tokenization 및 인덱싱
train_encodings = tokenizer(list(X_train), is_split_into_words=True, padding=True, truncation=True, return_tensors="pt")
eval_encodings = tokenizer(list(X_eval), is_split_into_words=True, padding=True, truncation=True, return_tensors="pt")

# 레이블을 인덱스로 변환
label_to_index = {'O': 0, 'B-PROFANITY': 1, 'I-PROFANITY': 2}
train_labels = [[label_to_index[label] for label in label_list] for label_list in Y_train]
eval_labels = [[label_to_index[label] for label in label_list] for label_list in Y_eval]

class ProfanityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# 데이터셋 생성
train_dataset = ProfanityDataset(train_encodings, train_labels)
eval_dataset = ProfanityDataset(eval_encodings, eval_labels)

# Trainer 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()